# 想要預測當日會不會下雨
featuers：
* 當日最低溫度
* 當日最低氣壓
* 濕度
* 風速

假設：
下雨條件只看這這四種，且發生時都在同一時間。

溫度 < 25(攝氏)

壓力 < 1010(hpa)

濕度 > 60(相對濕度)

風速 > 5(km/hr)

數值範圍：

溫度：15~30

氣壓：980~1030

濕度：30~80

風速：0~25

最多有 $10^6$ 種可能，全部條列太多了

所以用隨機產生 1000 個 data 去跑

In [1]:
import pandas as pd
import numpy
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz
import pydotplus 

In [2]:
temp_df = pd.DataFrame(numpy.random.randint(15,30,size=(1000, 1)), columns=['min_temp'])
press_df = pd.DataFrame(numpy.random.randint(980,1030,size=(1000, 1)), columns=['min_press'])
humidity_df = pd.DataFrame(numpy.random.randint(30,90,size=(1000, 1)), columns=['humidity'])
wind_df = pd.DataFrame(numpy.random.randint(0,25,size=(1000, 1)), columns=['wind_speed'])

In [3]:
df = temp_df
df.insert(1, 'min_press', press_df)
df.insert(2, 'humidity', humidity_df)
df.insert(3, 'wind_speed', wind_df)
df.head(10)

,min_temp,min_press,humidity,wind_speed
0,24,994,75,11
1,27,996,66,7
2,27,997,57,17
3,25,1003,74,9
4,27,1025,46,21
5,26,1027,81,7
6,25,1015,33,20
7,25,1014,47,16
8,18,999,43,8
9,22,999,81,22


In [4]:
ret = pd.DataFrame(numpy.zeros(1000), columns=['rain'])
for i in range(0, 999):
    if(df['min_temp'][i] < 25 and df['min_press'][i] < 1010 and df['humidity'][i] > 60 and df['wind_speed'][i] > 5):
        ret['rain'][i] = 1

In [5]:
ret.head(10)

,rain
0,1.0
1,0.0
2,0.0
3,0.0
4,0.0
5,0.0
6,0.0
7,0.0
8,0.0
9,1.0


In [6]:
df_train = df[:700]
df_train.head()

,min_temp,min_press,humidity,wind_speed
0,24,994,75,11
1,27,996,66,7
2,27,997,57,17
3,25,1003,74,9
4,27,1025,46,21


In [7]:
df_test = df[700:]
df_test.head()

,min_temp,min_press,humidity,wind_speed
700,28,995,40,5
701,16,1001,47,17
702,19,1011,55,2
703,28,1009,69,15
704,26,1002,55,20


In [8]:
y = ret['rain'].values

y_train = y[:700]
y_test = y[700:]

In [9]:
dtree=DecisionTreeClassifier(max_depth=6)
dtree.fit(df_train,y_train)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(df_train),
                class_names=['sun','rain'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

In [10]:
dtree.feature_importances_

array([0.30748032, 0.2290579 , 0.20802122, 0.25544055])

In [11]:
y_predict = dtree.predict(df_test)

y_predict

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1.,
       1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0.

In [12]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predict)

1.0

---

預測的結果是 $99\%$ 準確，猜測是上面條件設定過於準確，

這點看產生的 module 也可以發現，

上面的條件幾乎和我的假設一模一樣，

所以有這樣的準確率並不意外

若改成：條件符合時有 $80\%$ 機率會下雨

---

In [13]:
# 條件多加入 numpy.random.randint(0,4) != 0
# 接著重複上面的動作
ret_ran = pd.DataFrame(numpy.zeros(1000), columns=['rain'])
for i in range(0, 999):
    if(df['min_temp'][i] < 25 and df['min_press'][i] < 1010 and df['humidity'][i] > 60 and df['wind_speed'][i] > 5 and numpy.random.randint(0,4) != 0):
        ret_ran['rain'][i] = 1


In [14]:
df_ran_train = df[:700]
df_ran_test = df[700:]

y_ran = ret_ran['rain'].values

y_ran_train = y_ran[:700]
y_ran_test = y_ran[700:]

dtree=DecisionTreeClassifier(max_depth = 6)
dtree.fit(df_ran_train,y_ran_train)

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(df_train),
                class_names=['sun','rain'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree_ran.pdf")

True

In [15]:
dtree.feature_importances_

array([0.32099605, 0.21683194, 0.1969181 , 0.2652539 ])

In [16]:
y_ran_predict = dtree.predict(df_test)
from sklearn.metrics import accuracy_score
accuracy_score(y_ran_test, y_ran_predict)

0.98

---

看得出來即使有些 noise 其實還是有相當高的準確度，

而且做出來的 `decision tree` 和沒有 noise 的差不多，

有可能多出一些 leaf node

接著觀察 module 的部份發現會多了很多奇怪的條件，

tree 的長度長到設定的上限，

推測就是被 noise 影響的，

而最下面一些 leaf nodes 的 gini 值也不唯一，

然而若不設定 max_depth (即不設參數)

有可能會有 overfitting 的問題，準確率有機會降到不到 $90\%$

所以針對不同 dataset max_depth 應該是影響準確度的一個重要關鍵，

接著用 `random forest` 看看有什麼不同

----

In [17]:
from sklearn import ensemble, metrics
forest = ensemble.RandomForestClassifier(n_estimators = 100)
forest_fit = forest.fit(df_ran_train, y_ran_train)

test_y_predicted = forest.predict(df_test)

accuracy = metrics.accuracy_score(y_ran_test, test_y_predicted)
print(accuracy)

0.9666666666666667


---

若使用 Random Forest 似乎不設參數可以得到比較高的準確度，

若設定 max_depth 則得到的準確度會比不設還少，

與 decision tree 比較準確度兩者差不多(通常不會一樣)